In [152]:
import numpy as np
import matplotlib.pyplot as plt  
from sympy import *


## Symbols and constants 

In [153]:
gm1,gds1,gmb1,r1 = symbols('g_{m1} g_{ds1} g_{mb1} r_{1}')
gm2,gds2,gmb2,r2,rds2 = symbols('g_{m2} g_{ds2} g_{mb2} r_{2} r_{ds_2}')
AvLinGoal = np.power( 10,14/20 )
Vtn  = 0.6
Kn   = 200e-6
Lmin = 350e-9

av1 = (gm1+gmb1+gds1)/(gds1 + 1/(r1))


# Common Gate

### Common Gate Gain
$$A_v = \frac{v_o}{v_i}=\frac{g_m+g_{mb}+g_{ds}}{g_{ds}+1/r_1}$$

### Input Impedance

In [154]:
zin = 1/(gm1+gds1+gmb1-(gds1/av1))
print(latex(zin))



\frac{1}{- \frac{g_{ds1} \left(g_{ds1} + \frac{1}{r_{1}}\right)}{g_{ds1} + g_{m1} + g_{mb1}} + g_{ds1} + g_{m1} + g_{mb1}}


$$Z_{in}=\frac{1}{- \frac{g_{ds1} \left(g_{ds1} + \frac{1}{r_{1}}\right)}{g_{ds1} + g_{m1} + g_{mb1}} + g_{ds1} + g_{m1} + g_{mb1}}\approx\frac{1}{g_m+g_{mb}}$$

The input impedance should be $Z_{in} = 50 ~ \Omega$ 


With $g_{mb}\approx 0.2\cdot g_m$

In [155]:
eq_z1 = 1/(gm1*(1.2)) - 50
gm1 = solve(eq_z1,[gm1])[0]
print("gm = ",gm1)

gm =  0.0166666666666667


For the bias current 
$$g_m =\sqrt{2K_{n,p}\frac{W}{L}I_D}= \frac{2I_D}{V_{DSsat}}$$

Aiming for $V_{DSsat} = 100 ~\text{mV}$

In [156]:
Id1, vdsat1,W1,L1,Vgs1,r1 = symbols('I_{d1} V_{dsat1} W_1 L_1 V_{GS_1} R_1' )

eq_id1 = Eq(Id1,(Kn/2)*(W1/L1)*np.power(Vgs1-Vtn,2))

vdsat1 = 0.1

eq_gm1 = Eq(gm1, 2*Id1/vdsat1)

s = solve(eq_gm1,[Id1])
Id1 = s[0]
print("Id1: ",Id1*1e3,"mA")


Id1:  0.833333333333333 mA


Using $L_{min} = 350~\text{n}$ 
$$I_D=\frac{K_n}{2}\cdot\frac{W}{L}V_{DSsat}^2$$

In [157]:
wl1 = Id1/( 0.5*Kn*(vdsat1**2) )
print("W: ",wl1*Lmin*1e6,"um")

W:  291.666666666667 um


In [158]:
gds1 = 3.23e-04
gmb1 = 2.52e-03
av1 = (gm1+gmb1+gds1)/(gds1 + 1/(r1)) #Redefine in order to assume values

s = solve( av1-AvLinGoal,[r1] )
s
r1 = s[0]
print("R1 Value: ",r1)

R1 Value:  280.136349372232


# Common Source

Para $V_{DSsat} = 100~\text{mV}$

$$V_{DSsat} =V_{GS}-V_{Th}\Leftrightarrow V_{GS} = 0.7$$

$$A_v = −gm(rds//R_2)$$

$$gm = K_{n,p}\frac{W}{L}V_{DSsat}$$

In [ ]:
wl2,id2 = symbols("\\frac{W_2}{L_2} I_{d2}")
vdsat2 = 100e-3

rds2 = 1/(6.77e-08)

r2 = 1e3

av2 = gm2*( (rds2*r2)/(rds2+r2) )
eq_gm2 = Eq(Kn*wl2*vdsat2,gm2)
eq_id2 = Eq( Kn*0.5*wl2*np.power(vdsat2,2),id2 )
s = solve( [eq_gm2,eq_id2, av2-AvLinGoal],[id2,wl2,gm2,r2] )

print("Id2 = ",s[id2]*1e3,"mA")
print("W   = ",s[wl2]*Lmin*1e6,"um")
print("Gm2 = ",s[gm2])

Id2 =  0.894994168093703 mA
W   =  313.247958832796 um
Gm2 =  0.0178998833618741
